## **I. Import Libraries Necessary**

In [3]:
import re
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from my_functions.file_IO import *
from datetime import datetime
import time

## **II. Crawl Links of All Page In [Shopee Food](https://shopeefood.vn/ho-chi-minh/food/deals)**

#### 1. Open link https://shopeefood.vn/ho-chi-minh/food/deals

In [2]:
driver = webdriver.Chrome()
driver.maximize_window()
url = 'https://shopeefood.vn/ho-chi-minh/food/deals'
driver.get(url)
time.sleep(3)

#### 2. Extract number elements (number restaurants) on website.

In [ ]:
count_item = driver.find_element(by= By.CLASS_NAME, value="result")
count_item = count_item.text.split(' ')[0]

#### 3. Extract the corresponding detailed information links for each restaurant listed above.

In [4]:
i = 1
list_links = []

# Extract all elements (restaurants) from the first page.
div_list_restaurant = driver.find_elements(By.XPATH, '//div[@class="now-list-restaurant"]//div[@class="list-restaurant"]/div[@class="item-restaurant"]')
# Implement a loop to iterate through each element item within the div_list_restaurant class on the first page.
for div_restaurant in div_list_restaurant:
    # For each item in the PATH, utilize the find_element method to retrieve the hyperlink from the a tag. Extract the link from the href attribute and assign it to the link variable.
    link = div_restaurant.find_element(By.XPATH, 'a[@class="item-content"]').get_attribute('href')
    # Append the extracted link variable to the list_link to create a list of links for accessing each food item.
    list_links.append(link + '\n')
print('Page', i)
i += 1

# Employ a while loop to iterate through all remaining pages.
while True:
    try:
        # Click the "Next" button to navigate to the next page and wait for 10 seconds.
        next_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "span.icon.icon-paging-next"))
    )
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(4)
        print('Page', i)
        i += 1
        
        # Upon loading each new page, repeat the same actions as performed on the first page.
        div_list_restaurant = driver.find_elements(By.XPATH, '//div[@class="now-list-restaurant"]//div[@class="list-restaurant"]/div[@class="item-restaurant"]')
        # Implement a loop to iterate through each element item within the div_list_restaurant class.
        for div_restaurant in div_list_restaurant:
            # For each item in the PATH, utilize the find_element method to retrieve the hyperlink from the a tag. Extract the link from the href attribute and assign it to the link variable.
            try:
                link = div_restaurant.find_element(By.XPATH, 'a[@class="item-content"]').get_attribute('href')
                # Append the extracted link variable to the list_link to create a list of links for accessing each food item.
                list_links.append(link + '\n')
            except:
                list_links.append('link error' + '\n')
        
        # If the list of restaurant links matches the total number of restaurants on the website, terminate the process, save the link list to a txt file, and close the browser.
        if len(list_links) == int(count_item):
            write_file_txt('data_raw/txt/link_food_hcm.txt', list_links)
            driver.close()
            print('Done')
            break 
        else:
            continue
    except Exception as e:
        break

Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Page 10
Page 11
Page 12
Page 13
Page 14
Page 15
Page 16
Page 17
Page 18
Page 19
Page 20
Page 21
Page 22
Page 23
Page 24
Page 25
Page 26
Page 27
Page 28
Page 29
Page 30
Page 31
Page 32
Page 33
Page 34
Page 35
Page 36
Page 37
Page 38
Page 39
Page 40
Page 41
Page 42
Page 43
Page 44
Page 45
Page 46
Page 47
Page 48
Page 49
Page 50
Page 51
Page 52
Page 53
Page 54
Page 55
Page 56
Page 57
Page 58
Page 59
Page 60
Page 61
Page 62
Page 63
Page 64
Page 65
Page 66
Page 67
Page 68
Page 69
Page 70
Page 71
Page 72
Page 73
Page 74
Page 75
Page 76
Page 77
Page 78
Page 79
Page 80
Page 81
Page 82
Page 83
Page 84
Page 85
Page 86
Page 87
Page 88
Page 89
Page 90
Page 91
Page 92
Page 93
Page 94
Page 95
Page 96
Page 97
Page 98
Page 99
Page 100
Page 101
Page 102
Page 103
Page 104
Page 105
Page 106
Page 107
Page 108
Page 109
Page 110
Page 111
Page 112
Page 113
Page 114
Page 115
Page 116
Page 117
Page 118
Page 119
Page 120
Page 121
Page 122
Done


In [5]:
len(list_links)

3647

- The process successfully gathered 3647 restaurant links from the website.

## **III. Crawl Infomations Necessary**

#### 1. Build the function to extract informations necessary.

In [1]:
def get_data(list_link, restaurantid_start, reviewid_start):
    
    restaurant_id = restaurantid_start
    name_lst = []
    address_lst = []
    time_lst = []
    price_lst = []
    restaurantid_lst = []

    review_id = reviewid_start
    restaurant_review_lst = []
    user_review_lst = []
    time_review_lst = []
    rate_review_lst = []
    comment_review_lst = []
    reviewid_lst = []

    driver = webdriver.Chrome()
    driver.maximize_window()

    
    list_links = list_link

    for link in list_links:
        link = link.rstrip('\n')
        if link == 'link error':
            continue
        else:
            driver.get(link)
            time.sleep(2)
        
            # Restaurant ID
            restaurantid_lst.append(restaurant_id)
        
            # Restaurant Name
            try:
                name = driver.find_element(By.XPATH, '//div[@class="detail-restaurant-info"]/h1[@class="name-restaurant"]').text
                name_lst.append(name)
            except:
                name = ''
                name_lst.append(name)
            # Restaurant Address
            try:
                address = driver.find_element(By.XPATH, '//div[@class="detail-restaurant-info"]/div[@class="address-restaurant"]').text
                address_lst.append(address)
            except:
                address = ''
                address_lst.append(address)
            # Restaurant Time (Open - Close)
            try:
                _time = driver.find_element(By.XPATH, '//div[@class="detail-restaurant-info"]/div[@class="status-restaurant"]/div[@class="time"]').text
                time_lst.append(_time)
            except:
                _time = ''
                time_lst.append(_time)
            # Restaurant Price
            try:
                price = driver.find_element(By.XPATH, '//div[@class="detail-restaurant-info"]/div[@class="cost-restaurant"]').text
                price_lst.append(price)
            except:
                price = ''
                price_lst.append(price)
            
            
            # Review
            try:
                link_review = driver.find_element(By.XPATH, '//div[@class="detail-restaurant-info"]/div[@class="view-more-rating"]/a').get_attribute('href')
            except:
                link_review = ''
            else:
                driver_review = webdriver.Chrome()
                driver_review.maximize_window()
                driver_review.get(link_review)
        
                while True:
                    
                    try:
                        start = time.time()
                        WebDriverWait(driver_review, 20).until(EC.element_to_be_clickable((By.LINK_TEXT, "Xem thêm bình luận"))).click()
                        time.sleep(5)
                        end = time.time()
                        spend_time = end - start
                        if spend_time > 20:
                            break             
                    except:
                        break
                
                li_reviews = driver_review.find_elements(By.CLASS_NAME, value='review-item')
                for li_review in li_reviews:

                    # Review ID
                    reviewid_lst.append(review_id)
                    
                    # Reviewer
                    try:
                        review_user = li_review.find_element(By.CLASS_NAME, 'ru-username').text
                        user_review_lst.append(review_user)
                    except:
                        review_user = ''
                        user_review_lst.append(review_user)
                    
                    # Review Time
                    try:
                        review_time = li_review.find_element(By.CLASS_NAME, 'ru-time').text
                        time_review_lst.append(review_time)
                    except:
                        review_time = ''
                        time_review_lst.append(review_time)
        
                    # Review Rate
                    try:
                        review_rating = li_review.find_element(By.CLASS_NAME, 'review-points').text
                        rate_review_lst.append(review_rating)
                    except:
                        review_rating = 0
                        rate_review_lst.append(review_rating)
        
                    # Comment
                    try:
                        review_comment = li_review.find_element(By.CLASS_NAME, 'rd-des').text
                        comment_review_lst.append(review_comment)
                    except:
                        review_comment = ''
                        comment_review_lst.append(review_comment)  
        
                    # Restaurant ID of this Review
                    restaurant_review_lst.append(restaurant_id)
                    
                    review_id += 1           
                    
                else:
                    driver_review.close()
            restaurant_id += 1
    else:
        restaurant_df = pd.DataFrame({'RestaurantID' : restaurantid_lst,'Restaurant Name':name_lst, 'Address' : address_lst, 'Time' : time_lst, 'Price' : price_lst})
        restaurant_df.to_csv('data_raw/restaurant.csv')
        
        review_df = pd.DataFrame({'UserID': reviewid_lst,'User':user_review_lst, 'Review Time' : time_review_lst, 'Rating' : rate_review_lst, 'Comment' : comment_review_lst, 'RestaurantID': restaurant_review_lst})
        review_df.to_csv('data_raw/reviews.csv')
        
        driver.close()
    return restaurant_df , review_df
        

#### 2. Utilize the recently created function to retrieve the data.

In [5]:
list_links = read_file_txt('data_raw/link_food_hcm.txt')
restaurant_df , review_df = get_data(list_links, 1, 1)

#### 3. Read the two recently retrieved DataFrames and examine their basic information.

In [7]:
restaurant_df.head()

,Unnamed: 0,RestaurantID,Restaurant Name,Address,Time,Price
0,0,1,Bún Bò Đất Thánh - Shop Online,"221/16 Đất Thánh, P. 6, Tân Bình, TP. HCM",NaN,15.000 - 50.000
1,1,2,Bánh Mì Bảy Hổ - Phan Ngữ,"12 Phan Ngữ, P. Đa Kao, Quận 1, TP. HCM",06:00 - 20:00,10.000 - 30.000
2,2,3,Chân Gà Sốt Thái - Mr.Bon - Lê Bình,"100 Lê Bình, P. 4, Tân Bình, TP. HCM",06:00 - 23:00,10.000 - 100.000
3,3,4,Yumi Fruit - Trái Cây Sạch & Đồ Ăn Vặt - Shop ...,"150/19 Lê Thị Bạch Cát, P. 11, Quận 11, TP. HCM",00:00 - 23:59,20.000 - 100.000
4,4,5,Sinh Tố Tao Đàn - The Sun Avenue,"The Sun Avenue, Block SAV.06 - 00.01, 28 Mai C...",07:30 - 21:30,15.000 - 80.000


In [8]:
restaurant_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3647 entries, 0 to 3646
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       3647 non-null   int64 
 1   RestaurantID     3647 non-null   int64 
 2   Restaurant Name  3615 non-null   object
 3   Address          3615 non-null   object
 4   Time             2214 non-null   object
 5   Price            3615 non-null   object
dtypes: int64(2), object(4)
memory usage: 171.1+ KB


In [12]:
review_df.head()

,Unnamed: 0,UserID,User,Review Time,Rating,Comment,RestaurantID
0,0,1,Cang Pham,"31/03/2022 12:02:37,",5.4,Good\n...Xem thêm,1
1,1,2,NaN,NaN,NaN,NaN,1
2,2,3,NaN,NaN,NaN,NaN,1
3,3,4,NaN,NaN,0.0,NaN,1
4,4,5,Yumi Fruit Trái Cây Sạch & Đồ Ăn Vặt,7/11/2020 12:51,10.0,"Trái cây tươi và ngon, sẽ ủng hộ tiếp",4


In [13]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15729 entries, 0 to 15728
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    15729 non-null  int64  
 1   UserID        15729 non-null  int64  
 2   User          15642 non-null  object 
 3   Review Time   15643 non-null  object 
 4   Rating        15673 non-null  float64
 5   Comment       15640 non-null  object 
 6   RestaurantID  15729 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 860.3+ KB
